# DeepStream Launchpad Demo

Welcome to the DeepStream Launchpad Demo! In this Jupyter Notebook, you will learn how to build a Python application by constructing a Gstreamer pipeline using DeepStream elements and attaching a probe function to access meta data.

The diagram below describes the workflow of a typical DeepStream application.

![workflow](ds_workflow.png)


## Application Structure
The application consists primarily of the following parts:
1. Constructing and running **the pipeline**
2. **probe function**, to extract metadata from the buffer at some pipeline element

Some helper functions will also be described in this demo.

### The Pipeline 
This demonstration employs the following pipeline**:

![pipeline](ds_launchpad_pipeline.png)

The pipeline elements are described here:
* [`Gst-uridecodebin`](https://gstreamer.freedesktop.org/documentation/playback/uridecodebin.html?gi-language=python) - decode data from URI source into raw media
* [`Gst-nvstreammux`](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvstreammux.html) - stream muxer plugin, to form a batch of buffers from multiple input sources
* [`Gst-nvinfer`](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html) - NVIDIA TensorRT based plugin for primary and secondary (attribute classification of primary objects) detection and classification respectively
* [`Gst-nvtracker`](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvtracker.html) - OpenCV based tracker plugin (Gst-nvtracker) for object tracking with unique ID
* [`Gst-nvmultistreamtiler`](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvmultistreamtiler.html) - Multi Stream Tiler plugin for forming a 2D array of frames
*  [`Gst-nvdsosd`](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvdsosd.html) - Onscreen Display (OSD) plugin  to draw shaded boxes, rectangles and text on the composited frame using the generated metadata
* [`Gst-filesink`](https://gstreamer.freedesktop.org/documentation/coreelements/filesink.html?gi-language=python) - write stream data to a given file location

The app accepts one or more H.264/H.265 video streams as input. It creates
a source bin for each input and connects the bins to an instance of the
"nvstreammux" element, which forms the batch of frames. The batch of
frames is first fed to an "nvinfer" element, acting as the primary GPU inference engine (pgie), for batched inferencing. This is linked to an "nvtracker" instance which tracks the objects
detected by the pgie, which is followed by three more instances of "nvinfer", each performing secondary classification as secondary GPU inference engines (sgie). The batched buffer is then
composited into a 2D tile array using "nvmultistreamtiler." "nvdsosd" draws the bounding boxes and text from the metadata generated by the "nvinfer" elements, and the final output stream is saved to a file by the filesink.

**Note: a few additional elements are left out for simplicity, but will be described in detail in the application.

### The Probe Function
Each nvinfer element generates and attaches metadata to the Gst Buffer. Metadata starts at the batch level, created by the Gst-nvstreammux plugin, and subsidiary metadata structures can hold frame metadata, object metadata, and display metadata. See [DeepStream documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_metadata.html) for more details about metadata structure.

This means that by attaching a probe function at the end of the pipeline, at the sink of the tiler, we can retrieve the buffer and extract meaningful information about the objects from these inferences and customize some display parameters, such as the bounding box color of these objects.

Let's dive into some code!

## Step 1 - Import necessary libraries

First, we import necessary modules for this app. This of course, includes `pyds`, the module containing the DeepStream Python API.

In [ ]:
import sys
import os
import configparser
import math

import gi
gi.require_version('Gst', '1.0')
from gi.repository import GLib, Gst

import pyds

## Step 2 - Declare class label IDs
Define the class labels with their corresponding IDs according to the inference engine. To be used with an object counter in the probe function.

In [ ]:
PGIE_CLASS_ID_VEHICLE = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_PERSON = 2
PGIE_CLASS_ID_ROADSIGN = 3

## Step 3 - Declare tiler properties and OSD properties
These values will be used to set the width and height properties of the tiler, in pixels.
Use CPU process mode for OSD and display text in the output.

In [ ]:
TILED_OUTPUT_WIDTH=1920 # Tiler output width
TILED_OUTPUT_HEIGHT=1080 # Tiler output height

# NvOSD options
OSD_PROCESS_MODE= 0 # Process mode for NvOSD plugin. 0 for CPU, 1 for GPU
OSD_DISPLAY_TEXT= 1 # Enable/disable display of all text in NvOSD plugin. 0 for disable, 1 for enable

## Step 4 - Set file paths and additional options
Define the input stream file list, output file, and configuration files for GIEs and the tracker. We'll discuss the other options listed here in detail [later](#Simple-Customizations) in the notebook.

In [ ]:
# File paths
input_file_list = ["file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264", \
                   "file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264", \
                   "file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264", \
                   "file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"] 
                    # List of paths to input streams in URI format. Here we use 4 of the same stream as input.
output_file = "output.mp4" # Output file location
pgie_config_file = "./configs/dslaunchpad_pgie_config.txt" # Path to pgie config file
tracker_config_file = "./configs/dslaunchpad_tracker_config.txt" # Path to tracker config file
sgie1_config_file = "./configs/dslaunchpad_sgie1_config.txt" # Path to config file for first sgie
sgie2_config_file = "./configs/dslaunchpad_sgie2_config.txt" # Path to config file for second sgie
sgie3_config_file = "./configs/dslaunchpad_sgie3_config.txt" # Path to config file for third sgie

# Tracker options
enable_tracker = 1 # Enable/disable tracker and SGIEs. 0 for disable, 1 for enable

# Note: for colors, each Red/Green/Blue/Alpha (RGBA) value should be between 0.0 and 1.0, inclusive.
# Bounding box options
bbox_border_color = {"R": 1.0, "G": 0.0, "B": 0.0, "A": 1.0} # Color of bounding box. Set to red
bbox_has_bg_color = False # Bool for whether bounding box has background color
bbox_bg_color = {"R": 1.0, "G": 0.0, "B": 0.0, "A": 0.2} # Color of bbox background. Set to red with transparency

# Display text options, to be added to the frame.
text_x_offset = 10 # Offset in the x direction where string should appear
text_y_offset = 12 # Offset in the y direction where string should appear
text_font_name = "Serif" # Font name
text_font_size = 10 # Font size
text_font_color = {"R" : 1.0, "G": 1.0, "B": 1.0, "A": 1.0} # Color of text font. Set to white
text_set_bg_color = True # Bool for whether text box has background color
text_bg_color = {"R": 0.0, "G": 0.0, "B": 0.0, "A": 1.0} # Color of text box background. Set to black

## Step 5 - Define the probe function
Let's write our probe function, to be attached at the sink pad of the tiler.

This probe function will retrieve the Gst buffer, then retrieve the batch level metadata from the buffer. From the batch level metadata, we retrieve a list of frame metadata (one frame meta for each input stream), and from each frame metadata, we retrieve a list of object metadata, which were attached to the frame by the GIEs.

For each object in a frame, we can apply customizations, such as setting the color of the bounding box and applying a background color to the bbox.

For each frame, we can add text to the display by acquiring and attaching a display meta object. We set the text to be displayed and customize parameters such as the position of the text, the font, and the text box.

In [ ]:
def tiler_sink_pad_buffer_probe(pad,info,u_data):
    frame_number=0
    #Intiallizing object counter with 0.
    obj_counter = {
        PGIE_CLASS_ID_VEHICLE:0,
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_ROADSIGN:0
    }
    num_rects=0
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve batch metadata from the gst_buffer
    # Note that pyds.gst_buffer_get_nvds_batch_meta() expects the
    # C address of gst_buffer as input, which is obtained with hash(gst_buffer)
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            # Note that l_frame.data needs a cast to pyds.NvDsFrameMeta
            # The casting is done by pyds.NvDsFrameMeta.cast()
            # The casting also keeps ownership of the underlying memory
            # in the C code, so the Python garbage collector will leave
            # it alone.
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break

        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        while l_obj is not None:
            try:
                # Casting l_obj.data to pyds.NvDsObjectMeta
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            obj_counter[obj_meta.class_id] += 1 # Increment object counter
            rectparams = obj_meta.rect_params # Retrieve bounding box parameters
            # Set color of bbox. (red, green, blue, alpha), 0.0 <= RGBA <= 1.0
            rectparams.border_color.set(bbox_border_color["R"], bbox_border_color["G"], bbox_border_color["B"], bbox_border_color["A"])
            # Set whether bbox has background color
            rectparams.has_bg_color = bbox_has_bg_color
            # If bbox has background color, set background color
            if bbox_has_bg_color:
                rectparams.bg_color.set(bbox_bg_color["R"], bbox_bg_color["G"], bbox_bg_color["B"], bbox_bg_color["A"])
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break

        # Acquiring a display meta object. The memory ownership remains in
        # the C code so downstream plugins can still access it. Otherwise
        # the garbage collector will claim it when this probe function exits.
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Setting display text to be shown on screen
        # Note that the pyds module allocates a buffer for the string, and the
        # memory will not be claimed by the garbage collector.
        # Reading the display_text field here will return the C address of the
        # allocated string. Use pyds.get_string() to get the string content.
        py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])

        # Now set the offsets where the string should appear
        py_nvosd_text_params.x_offset = text_x_offset
        py_nvosd_text_params.y_offset = text_y_offset

        # Font, font-color and font-size
        py_nvosd_text_params.font_params.font_name = text_font_name
        py_nvosd_text_params.font_params.font_size = text_font_size
        # set(red, green, blue, alpha); set to White
        py_nvosd_text_params.font_params.font_color.set(text_font_color["R"], text_font_color["G"], text_font_color["B"], text_font_color["A"])

        # Text background color
        py_nvosd_text_params.set_bg_clr = text_set_bg_color
        # set(red, green, blue, alpha); set to Black
        py_nvosd_text_params.text_bg_clr.set(text_bg_color["R"], text_bg_color["G"], text_bg_color["B"], text_bg_color["A"])
        # Using pyds.get_string() to get display_text as string
        print(pyds.get_string(py_nvosd_text_params.display_text))
        # Add the display meta to the frame
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    
    return Gst.PadProbeReturn.OK	


## Step 6 - Define helper functions for creating source bins

To allow for any video format, we use GStreamer's [uridecodebin](https://gstreamer.freedesktop.org/documentation/playback/uridecodebin.html?gi-language=python), which will automatically resolve the type of source element needed for the given URI and connect it to a decoder.

Since there are properties we may want to set based on the type of source, we write callback functions to connect to certain signals. The `create_source_bin` helper function takes an index and a URI and creates a uridecodebin element, then connects the callback functions to the signals of interest.

In this case, we have two callback functions. We connect `cb_newpad` to the "pad-added" signal, so that every time a pad is added (for sources with both audio and video, a pad is added for each), we check if the pad is for video, and if it is, we link the decodebin only if it has picked an nvidia decoder plugin. We connect `decodebin_child_added` to the "child-added" signal. When a child is added, if it is a [`decodebin`](https://gstreamer.freedesktop.org/documentation/playback/decodebin.html?gi-language=python) element (used internally by the uridecodebin), then we connect the callback to the "child-added" signal of that decodebin. If the child added was a source, then we check if the source has the `drop-on-latency` property and set to true if it does.

In [ ]:
def cb_newpad(decodebin, decoder_src_pad,data):
    print("In cb_newpad\n")
    caps=decoder_src_pad.get_current_caps()
    if not caps:
        caps = decoder_src_pad.query_caps()
    gststruct=caps.get_structure(0)
    gstname=gststruct.get_name()
    source_bin=data
    features=caps.get_features(0)

    # Need to check if the pad created by the decodebin is for video and not
    # audio.
    print("gstname=",gstname)
    if(gstname.find("video")!=-1):
        # Link the decodebin pad only if decodebin has picked nvidia
        # decoder plugin nvdec_*. We do this by checking if the pad caps contain
        # NVMM memory features.
        print("features=",features)
        if features.contains("memory:NVMM"):
            # Get the source bin ghost pad
            bin_ghost_pad=source_bin.get_static_pad("src")
            if not bin_ghost_pad.set_target(decoder_src_pad):
                sys.stderr.write("Failed to link decoder src pad to source bin ghost pad\n")
        else:
            sys.stderr.write(" Error: Decodebin did not pick nvidia decoder plugin.\n")

def decodebin_child_added(child_proxy,Object,name,user_data):
    print("Decodebin child added:", name, "\n")
    # Connect callback to internal decodebin signal
    if(name.find("decodebin") != -1):
        Object.connect("child-added",decodebin_child_added,user_data)

    if "source" in name:
        source_element = child_proxy.get_by_name("source")
        if source_element.find_property('drop-on-latency') != None:
            Object.set_property("drop-on-latency", True)

def create_source_bin(index,uri):
    print("Creating source bin")

    # Create a source GstBin to abstract this bin's content from the rest of the
    # pipeline
    bin_name="source-bin-%02d" %index
    print(bin_name)
    nbin=Gst.Bin.new(bin_name)
    if not nbin:
        sys.stderr.write(" Unable to create source bin \n")

    # Source element for reading from the uri.
    # We will use decodebin and let it figure out the container format of the
    # stream and the codec and plug the appropriate demux and decode plugins.
    uri_decode_bin=Gst.ElementFactory.make("uridecodebin", "uri-decode-bin")
    if not uri_decode_bin:
        sys.stderr.write(" Unable to create uri decode bin \n")
    # We set the input uri to the source element
    uri_decode_bin.set_property("uri",uri)
    # Connect to the "pad-added" signal of the decodebin which generates a
    # callback once a new pad for raw data has been created by the decodebin
    uri_decode_bin.connect("pad-added",cb_newpad,nbin)
    uri_decode_bin.connect("child-added",decodebin_child_added,nbin)

    # We need to create a ghost pad for the source bin which will act as a proxy
    # for the video decoder src pad. The ghost pad will not have a target right
    # now. Once the decode bin creates the video decoder and generates the
    # cb_newpad callback, we will set the ghost pad target to the video decoder
    # src pad.
    Gst.Bin.add(nbin,uri_decode_bin)
    bin_pad=nbin.add_pad(Gst.GhostPad.new_no_target("src",Gst.PadDirection.SRC))
    if not bin_pad:
        sys.stderr.write(" Failed to add ghost pad in source bin \n")
        return None
    return nbin

## Step 7 - Define bus call function
Each pipeline contains a bus, which handles forwarding messages from streaming threads to the application. A message handler must be set, which will periodically check for new messages and call the callback function when a message is available. We define the callback below to handle end-of-stream (EOS) messages, warnings, and errors.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        loop.quit()
    elif t==Gst.MessageType.WARNING:
        err, debug = message.parse_warning()
        sys.stderr.write("Warning: %s: %s\n" % (err, debug))
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        loop.quit()
    return True

## Step 8 - Build and run pipeline

## Step 8-1 - Initialize GStreamer
Here, we begin construction of the pipeline. We start by following the standard GStreamer initialization procedure.

In [ ]:
# Standard initialization procedure
Gst.init(None)

# Create gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")
    
number_sources = len(input_file_list) # Declare number of input sources

## Step 8-2 - Create pipeline elements
We create each pipeline element, set their respective properties, and add them to the pipeline.

### Step 8-2-1 - Create the streammux

The Gst-nvstreammux plugin combines `batch-size` number of frames from multiple input sources into a batched buffer. This batch is pushed downstream when the batch is filled or the `batched-push-timeout` time limit is reached. All frames in the batch are scaled to the same resolution, specified by the `width` and `height` properties in pixels. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvstreammux.html) for more details and other properties.

In [ ]:
print("Creating streammux \n ")

# Create nvstreammux instance to form batches from one or more sources.
streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
if not streammux:
    sys.stderr.write(" Unable to create NvStreamMux \n")

# Set streammux properties
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', number_sources)
streammux.set_property('batched-push-timeout', 4000000)

pipeline.add(streammux) # Add streammux to the pipeline

### Step 8-2-2 - Create a source bin for each input file
We call our previously defined helper function for creating a source bin for each input file, and add it to the pipeline. The sink pad of the streammux is retrieved, and each source is linked to a sink pad of the streammux.

In [ ]:
# Iterate through each input source
for i in range(number_sources):
    print("Creating source_bin ",i," \n ")
    uri_name=input_file_list[i] # Retrieve input file URI
    if uri_name.find("rtsp://") == 0 :
        is_live = True
    source_bin=create_source_bin(i, uri_name) # Call helper to create source bin
    if not source_bin:
        sys.stderr.write("Unable to create source bin \n")
    pipeline.add(source_bin) # Add source bin to pipeline
    padname="sink_%u" %i
    sinkpad= streammux.get_request_pad(padname) # Retrieve a sink pad from the streammux element
    if not sinkpad:
        sys.stderr.write("Unable to create sink pad bin \n")
    srcpad=source_bin.get_static_pad("src") # Retrieve the source pad of the source bin
    if not srcpad:
        sys.stderr.write("Unable to create src pad bin \n")
    srcpad.link(sinkpad) # Link the source pad of the source bin to the sink pad of the streammux

### Step 8-2-3 - Create nvinfer element as PGIE
The Gst-nvinfer plugin takes the batched buffer from the streammux and performs inference using NVIDIA TensorRT. We create this element to perform inference in primary mode. Behaviour of the inference are set through properties in the configuration file. In this case, we are using ResNet10 to detect objects and classify them as Person, Vehicle, Bicycle, or Roadsign.

In [ ]:
print("Creating nvinfer (PGIE) \n ")

# Create nvinfer element
pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
if not pgie:
    sys.stderr.write(" Unable to create pgie \n")
    
# Set nvinfer properties
pgie.set_property('config-file-path', pgie_config_file)
pgie_batch_size=pgie.get_property("batch-size")

if(pgie_batch_size != number_sources):
    print("WARNING: Overriding infer-config batch-size",pgie_batch_size," with number of sources ", number_sources," \n")
    pgie.set_property("batch-size",number_sources)


# Add pgie to pipeline
pipeline.add(pgie)

Let's take a quick look at the contents of the pgie configuration file. These configurations determine the behavior of the plugin, such as the model used for inference, the process mode (primary or secondary), etc. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvinfer.html#gst-nvinfer-file-configuration-specifications) for more details on file configuration specifications.
```bash
[property]
gpu-id=0 # Device ID of GPU to use for inference
net-scale-factor=0.0039215697906911373 # Pixel scaling factor
model-file=../../../../samples/models/Primary_Detector/resnet10.caffemodel # Path to caffemodel file
proto-file=../../../../samples/models/Primary_Detector/resnet10.prototxt # Path to prototxt file
# Path toserialized model engine file
model-engine-file=../../../../samples/models/Primary_Detector/resnet10.caffemodel_b1_gpu0_int8.engine
labelfile-path=../../../../samples/models/Primary_Detector/labels.txt # Path to text file containing labels
int8-calib-file=../../../../samples/models/Primary_Detector/cal_trt.bin # Path to calibration file
force-implicit-batch-dim=1 # Force the implicit batch dimension mode
batch-size=1 # Number of frames/objects to be inferred together in a batch
network-mode=1 # Data format to be used by inference - 0: FP32, 1: INT8, 2: FP16
num-detected-classes=4 # Number of classes detected by the network
interval=0 # Number of consecutive batches to be skipped for inference
gie-unique-id=1 # Unique ID to assign to GIE
output-blob-names=conv2d_bbox;conv2d_cov/Sigmoid # Array of output layer names
#scaling-filter=0
#scaling-compute-hw=0

[class-attrs-all]
pre-cluster-threshold=0.2 # Detection threshold to be applied prior to clustering
eps=0.2 # Epsilon values for OpenCV grouprectangles() function and DBSCAN algorithm
group-threshold=1 # Threshold value for rectangle merging for OpenCV grouprectangles() function
```

### Step 8-2-4 - Create tracker, if enabled
The Gst-nvtracker plugin uses a low-level tracker library (NvDCF in our case) to track the objects detected by the PGIE with persistent IDs over time. Properties for the tracker are also set through a configuration file. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvtracker.html) for more details

In [ ]:
if enable_tracker:
    print("Creating nvtracker\n ")
    
    # Create nvtracker element
    tracker = Gst.ElementFactory.make("nvtracker", "tracker")
    if not tracker:
        sys.stderr.write(" Unable to create tracker \n")

    # Parse tracker config file and set properties
    config = configparser.ConfigParser()
    config.read(tracker_config_file)
    config.sections()

    for key in config['tracker']:
        if key == 'tracker-width' :
            tracker_width = config.getint('tracker', key)
            tracker.set_property('tracker-width', tracker_width)
        if key == 'tracker-height' :
            tracker_height = config.getint('tracker', key)
            tracker.set_property('tracker-height', tracker_height)
        if key == 'gpu-id' :
            tracker_gpu_id = config.getint('tracker', key)
            tracker.set_property('gpu_id', tracker_gpu_id)
        if key == 'll-lib-file' :
            tracker_ll_lib_file = config.get('tracker', key)
            tracker.set_property('ll-lib-file', tracker_ll_lib_file)
        if key == 'll-config-file' :
            tracker_ll_config_file = config.get('tracker', key)
            tracker.set_property('ll-config-file', tracker_ll_config_file)

    # Add tracker to pipeline
    pipeline.add(tracker)

### Step 8-2-5 - Create SGIEs, if tracker enabled
If the tracker is enabled, we create three more Gst-nvinfer elements to perform inference in secondary mode. These SGIEs will perform inference on the tracked objects from the upstream nvtracker element and classify each vehicle based on car make, car color, and car type, respectively to each SGIE.

In [ ]:
if enable_tracker:
    print("Creating nvinfer (SGIE1) \n ")
    
    # Create nvinfer element
    sgie1 = Gst.ElementFactory.make("nvinfer", "secondary1-nvinference-engine")
    if not sgie1:
        sys.stderr.write(" Unable to make sgie1 \n")

    print("Creating nvinfer (SGIE2) \n ")
    
    # Create nvinfer element
    sgie2 = Gst.ElementFactory.make("nvinfer", "secondary2-nvinference-engine")
    if not sgie2:
        sys.stderr.write(" Unable to make sgie2 \n")
        
    print("Creating nvinfer (SGIE3) \n ")
    
    # Create nvinfer element
    sgie3 = Gst.ElementFactory.make("nvinfer", "secondary3-nvinference-engine")
    if not sgie3:
        sys.stderr.write(" Unable to make sgie3 \n")

    # Set config file for each SGIE
    sgie1.set_property('config-file-path', sgie1_config_file)
    sgie2.set_property('config-file-path', sgie2_config_file)
    sgie3.set_property('config-file-path', sgie3_config_file)

    # Add SGIEs to pipeline
    pipeline.add(sgie1)
    pipeline.add(sgie2)
    pipeline.add(sgie3)

### Step 8-2-6 - Create multistreamtiler
The Gst-nvmultistreamtiler plugin takes the batched buffer from upstream and composites the streams into a `rows` x `columns` 2D tile in row-major order based on stream IDs. The 2D tile will have `width` x `height` resolution. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvmultistreamtiler.html) for details and additional properties.

In [ ]:
print("Creating tiler \n ")

# Create nvmultistreamtiler element
tiler=Gst.ElementFactory.make("nvmultistreamtiler", "nvtiler")
if not tiler:
    sys.stderr.write(" Unable to create tiler \n")

# Calculate number of rows and columns based on number of input sources
tiler_rows=int(math.sqrt(number_sources))
tiler_columns=int(math.ceil((1.0*number_sources)/tiler_rows))

# Set tiler properties
tiler.set_property("rows",tiler_rows)
tiler.set_property("columns",tiler_columns)
tiler.set_property("width", TILED_OUTPUT_WIDTH)
tiler.set_property("height", TILED_OUTPUT_HEIGHT)

# Add tiler to pipeline
pipeline.add(tiler)

### Step 8-2-7 - Create nvvideoconvert to convert from NV12 to RGBA as required by nvdsosd
The Gst-nvvideoconvert plugin performs video color format conversion. The nvdsosd plugin, next in the pipeline, requires RGBA format, which nvvideoconvert plugin outputs. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvvideoconvert.html) for more details.

In [ ]:
print("Creating nvvidconv \n ")

# Create nvvideoconvert element
nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
if not nvvidconv:
    sys.stderr.write(" Unable to create nvvidconv \n")

# Add nvvideoconvert to pipeline
pipeline.add(nvvidconv)

### Step 8-2-8 - Create OSD to draw on the converted RGBA buffer
The Gst-nvdsosd plugin accepts an RGBA buffer from upstream and draws the boudning boxes and text given the attached metadata. The text and bounding box parameters are configurable through the metadata, which we customize in the probe function. The `process-mode` property indicates whether CPU mode or GPU mode is used to the draw the objects, and the `display-text` property indicates whether or not to display all text. See the DeepStream [documentation](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_plugin_gst-nvdsosd.html) for more details and properties.

In [ ]:
print("Creating nvosd \n ")

# Create nvdsosd element
nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
if not nvosd:
    sys.stderr.write(" Unable to create nvosd \n")
    
# Set nvosd properties
nvosd.set_property('process-mode',OSD_PROCESS_MODE)
nvosd.set_property('display-text',OSD_DISPLAY_TEXT)

# Add nvosd to pipeline
pipeline.add(nvosd)

### Step 8-2-9 - Create set of nvvidconv, encoder, mux, and parser to prepare stream for filesink
The nvvideoconvert element will convert the buffer to the color format accepted by the nvv4l2h264enc element, which will encode the buffer into h264 video format. The [qtmux](https://gstreamer.freedesktop.org/documentation/isomp4/qtmux.html?gi-language=python) will merge any audio into the video before passing the buffer to the [h264parse](https://gstreamer.freedesktop.org/documentation/videoparsersbad/h264parse.html?gi-language=python), which will parse the recently encoded h264 stream to prepare for writing to the filesink.

In [ ]:
print("Creating nvvidconv \n ")

# Create nvvideoconvert element
nvvidconv2 = Gst.ElementFactory.make("nvvideoconvert", "convertor2")
if not nvvidconv2:
    sys.stderr.write(" Unable to create nvvidconv2 \n")

print("Creating nvv4l2h264enc \n ")

# Create nvv4l2h264enc element
encoder = Gst.ElementFactory.make("nvv4l2h264enc", "encoder")
if not encoder:
    sys.stderr.write(" Unable to create encoder \n")

print("Creating qtmux \n ")

# Create qtmux element
mux = Gst.ElementFactory.make("qtmux", "muxer")
if not mux:
    sys.stderr.write(" Unable to create muxer \n")

# Create h264parse element
print("Creating h264parse\n ")
parser1 = Gst.ElementFactory.make("h264parse", "h264-parser2")
if not parser1:
    sys.stderr.write(" Unable to create parser1 \n")

# Add all to pipeline
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(mux)
pipeline.add(parser1)

### Step 8-2-10 - Create filesink to save output stream to file
The filesink element writes the buffer to a file given the `location`. See the Gstreamer [documentation](https://gstreamer.freedesktop.org/documentation/coreelements/filesink.html?gi-language=python) for more details.

In [ ]:
print("Creating FileSink \n")

# Create filesink
sink = Gst.ElementFactory.make("filesink", "nvvideo-renderer")

# Set filesink properties
sink.set_property('location', output_file)
sink.set_property("sync",0) # Don't sync to clock, to allow samples to be played as fast as possible

# Add sink to pipeline
pipeline.add(sink)

## Step 8-3 - Link the pipeline elements
After we construct the elements and add them to the pipeline, they must be linked to each other in the proper order.

In [ ]:
# Note that the sources were already linked to the streammux
# Link streammux > pgie
streammux.link(pgie)
# If tracker is enabled, link pgie > tracker > sgie1 > sgie2 > sgie3 > tiler
if enable_tracker:
    pgie.link(tracker)
    tracker.link(sgie1)
    sgie1.link(sgie2)
    sgie2.link(sgie3)
    sgie3.link(tiler)
# Otherwise, linke pgie > tiler
else:
    pgie.link(tiler)
# Link tiler > nvvidconv > nvosd > nvvidconv2 > encoder > parser1 > mux > sink
tiler.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(parser1)
parser1.link(mux)
mux.link(sink)

## Step 8-4 - Create an event loop and bus messages to it
We first create a mainloop, which continuously checks for occurrences on its watch. We add the bus signal to the watch and connect the previously defined callback function to the loop. This way, when a message is available on the bus, the callback function will be called to handle the message.

In [ ]:
loop = GLib.MainLoop() # Create a mainloop
bus = pipeline.get_bus() # Retrieve the bus from the pipeline
bus.add_signal_watch() # Add a watch for new messages on the bus
bus.connect ("message", bus_call, loop) # Connect the loop to the callback function

## Step 8-5 - Attach the probe function to extract the meta data generated
We want the probe to be attached at an element after all GIEs to be able access the attached metadata, but before the buffer passes through the tiler, since it combines all frames. So we attach the probe to the sink of the tiler.

In [ ]:
# We add probe to the sink pad of the tiler element, since by that time, 
# the buffer would have had gotten all the metadata.

# Retrieve the sink pad of the tiler
tilersinkpad = tiler.get_static_pad("sink")
if not tilersinkpad:
    sys.stderr.write(" Unable to get sink pad of tiler \n")

# Add probe function to the sink pad of the tiler
tilersinkpad.add_probe(Gst.PadProbeType.BUFFER, tiler_sink_pad_buffer_probe, 0)

## Step 8-6 - Run the pipline
Note: this step will take several minutes to finish running. Do not execute the next cell until this step is complete, i.e. you see the text `<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>` after a series of `Frame Number = ... Number of Objects = ... Vehicle_count = ... Person_count = ...` prints and `End-of-stream` is reached.

In [ ]:
print("Starting pipeline \n")
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# cleaning up as the pipeline comes to an end
pipeline.set_state(Gst.State.NULL)

## Step 9 - View the output written by filesink

Observe the output below. In the output video, you should see four streams corresponding to each input source tiled in a 2 x 2 matrix. At the top left of each tiled stream, there should be display text descrbing the frame number, number of objects, vehicle count, and person count for each frame. A red bounding box should appear around each detected object, along with a text label describing the object (Person, Car, etc.), and the car labels should also include information from the SGIEs, i.e. car make, type, and color.

**Note: if the video does not fit into the Jupyter Notebook cell, you can scroll to the right in the cell and click the fullscreen button to view the video in its full resolution. 

In [ ]:
from IPython.display import Video

Video("output.mp4")

### Congratulations! 
You've run your first Deepstream Python app. Now that you've seen the pipeline code broken down, let's put it into a function so we can run it easily.

In [ ]:
def build_and_run_pipeline():
    # Standard initialization procedure
    Gst.init(None)

    # Create gstreamer elements
    # Create Pipeline element that will form a connection of other elements
    print("Creating Pipeline \n ")
    pipeline = Gst.Pipeline()

    if not pipeline:
        sys.stderr.write(" Unable to create Pipeline \n")

    number_sources = len(input_file_list) # Declare number of input sources
    
    print("Creating streammux \n ")

    # Create nvstreammux instance to form batches from one or more sources.
    streammux = Gst.ElementFactory.make("nvstreammux", "Stream-muxer")
    if not streammux:
        sys.stderr.write(" Unable to create NvStreamMux \n")

    # Set streammux properties
    streammux.set_property('width', 1920)
    streammux.set_property('height', 1080)
    streammux.set_property('batch-size', number_sources)
    streammux.set_property('batched-push-timeout', 4000000)

    pipeline.add(streammux) # Add streammux to the pipeline
    
    # Iterate through each input source
    for i in range(number_sources):
        print("Creating source_bin ",i," \n ")
        uri_name=input_file_list[i] # Retrieve input file URI
        if uri_name.find("rtsp://") == 0 :
            is_live = True
        source_bin=create_source_bin(i, uri_name) # Call helper to create source bin
        if not source_bin:
            sys.stderr.write("Unable to create source bin \n")
        pipeline.add(source_bin) # Add source bin to pipeline
        padname="sink_%u" %i
        sinkpad= streammux.get_request_pad(padname) # Retrieve a sink pad from the streammux element
        if not sinkpad:
            sys.stderr.write("Unable to create sink pad bin \n")
        srcpad=source_bin.get_static_pad("src") # Retrieve the source pad of the source bin
        if not srcpad:
            sys.stderr.write("Unable to create src pad bin \n")
        srcpad.link(sinkpad) # Link the source pad of the source bin to the sink pad of the streammux
        
        print("Creating nvinfer (PGIE) \n ")

    # Create nvinfer element
    pgie = Gst.ElementFactory.make("nvinfer", "primary-inference")
    if not pgie:
        sys.stderr.write(" Unable to create pgie \n")

    # Set nvinfer properties
    pgie.set_property('config-file-path', pgie_config_file)
    pgie_batch_size=pgie.get_property("batch-size")

    if(pgie_batch_size != number_sources):
        print("WARNING: Overriding infer-config batch-size",pgie_batch_size," with number of sources ", number_sources," \n")
        pgie.set_property("batch-size",number_sources)


    # Add pgie to pipeline
    pipeline.add(pgie)
    
    if enable_tracker:
        print("Creating nvtracker\n ")

        # Create nvtracker element
        tracker = Gst.ElementFactory.make("nvtracker", "tracker")
        if not tracker:
            sys.stderr.write(" Unable to create tracker \n")

        # Parse tracker config file and set properties
        config = configparser.ConfigParser()
        config.read(tracker_config_file)
        config.sections()

        for key in config['tracker']:
            if key == 'tracker-width' :
                tracker_width = config.getint('tracker', key)
                tracker.set_property('tracker-width', tracker_width)
            if key == 'tracker-height' :
                tracker_height = config.getint('tracker', key)
                tracker.set_property('tracker-height', tracker_height)
            if key == 'gpu-id' :
                tracker_gpu_id = config.getint('tracker', key)
                tracker.set_property('gpu_id', tracker_gpu_id)
            if key == 'll-lib-file' :
                tracker_ll_lib_file = config.get('tracker', key)
                tracker.set_property('ll-lib-file', tracker_ll_lib_file)
            if key == 'll-config-file' :
                tracker_ll_config_file = config.get('tracker', key)
                tracker.set_property('ll-config-file', tracker_ll_config_file)

        # Add tracker to pipeline
        pipeline.add(tracker)
        
        print("Creating nvinfer (SGIE1) \n ")
    
        # Create nvinfer element
        sgie1 = Gst.ElementFactory.make("nvinfer", "secondary1-nvinference-engine")
        if not sgie1:
            sys.stderr.write(" Unable to make sgie1 \n")

        print("Creating nvinfer (SGIE2) \n ")

        # Create nvinfer element
        sgie2 = Gst.ElementFactory.make("nvinfer", "secondary2-nvinference-engine")
        if not sgie2:
            sys.stderr.write(" Unable to make sgie2 \n")

        print("Creating nvinfer (SGIE3) \n ")

        # Create nvinfer element
        sgie3 = Gst.ElementFactory.make("nvinfer", "secondary3-nvinference-engine")
        if not sgie3:
            sys.stderr.write(" Unable to make sgie3 \n")

        # Set config file for each SGIE
        sgie1.set_property('config-file-path', sgie1_config_file)
        sgie2.set_property('config-file-path', sgie2_config_file)
        sgie3.set_property('config-file-path', sgie3_config_file)

        # Add SGIEs to pipeline
        pipeline.add(sgie1)
        pipeline.add(sgie2)
        pipeline.add(sgie3)
        
    print("Creating tiler \n ")

    # Create nvmultistreamtiler element
    tiler=Gst.ElementFactory.make("nvmultistreamtiler", "nvtiler")
    if not tiler:
        sys.stderr.write(" Unable to create tiler \n")

    # Calculate number of rows and columns based on number of input sources
    tiler_rows=int(math.sqrt(number_sources))
    tiler_columns=int(math.ceil((1.0*number_sources)/tiler_rows))

    # Set tiler properties
    tiler.set_property("rows",tiler_rows)
    tiler.set_property("columns",tiler_columns)
    tiler.set_property("width", TILED_OUTPUT_WIDTH)
    tiler.set_property("height", TILED_OUTPUT_HEIGHT)

    # Add tiler to pipeline
    pipeline.add(tiler)
    
    print("Creating nvvidconv \n ")

    # Create nvvideoconvert element
    nvvidconv = Gst.ElementFactory.make("nvvideoconvert", "convertor")
    if not nvvidconv:
        sys.stderr.write(" Unable to create nvvidconv \n")

    # Add nvvideoconvert to pipeline
    pipeline.add(nvvidconv)
    
    print("Creating nvosd \n ")

    # Create nvdsosd element
    nvosd = Gst.ElementFactory.make("nvdsosd", "onscreendisplay")
    if not nvosd:
        sys.stderr.write(" Unable to create nvosd \n")

    # Set nvosd properties
    nvosd.set_property('process-mode',OSD_PROCESS_MODE)
    nvosd.set_property('display-text',OSD_DISPLAY_TEXT)

    # Add nvosd to pipeline
    pipeline.add(nvosd)
    
    print("Creating nvvidconv \n ")

    # Create nvvideoconvert element
    nvvidconv2 = Gst.ElementFactory.make("nvvideoconvert", "convertor2")
    if not nvvidconv2:
        sys.stderr.write(" Unable to create nvvidconv2 \n")

    print("Creating nvv4l2h264enc \n ")

    # Create nvv4l2h264enc element
    encoder = Gst.ElementFactory.make("nvv4l2h264enc", "encoder")
    if not encoder:
        sys.stderr.write(" Unable to create encoder \n")

    print("Creating qtmux \n ")

    # Create qtmux element
    mux = Gst.ElementFactory.make("qtmux", "muxer")
    if not mux:
        sys.stderr.write(" Unable to create muxer \n")

    # Create h264parse element
    print("Creating h264parse\n ")
    parser1 = Gst.ElementFactory.make("h264parse", "h264-parser2")
    if not parser1:
        sys.stderr.write(" Unable to create parser1 \n")

    # Add all to pipeline
    pipeline.add(nvvidconv2)
    pipeline.add(encoder)
    pipeline.add(mux)
    pipeline.add(parser1)
    
    print("Creating FileSink \n")

    # Create filesink
    sink = Gst.ElementFactory.make("filesink", "nvvideo-renderer")

    # Set filesink properties
    sink.set_property('location', output_file)
    sink.set_property("sync",0)

    # Add sink to pipeline
    pipeline.add(sink)
    
    # Note that the sources were already linked to the streammux
    # Link streammux > pgie
    streammux.link(pgie)
    # If tracker is enabled, link pgie > tracker > sgie1 > sgie2 > sgie3 > tiler
    if enable_tracker:
        pgie.link(tracker)
        tracker.link(sgie1)
        sgie1.link(sgie2)
        sgie2.link(sgie3)
        sgie3.link(tiler)
    # Otherwise, linke pgie > tiler
    else:
        pgie.link(tiler)
    # Link tiler > nvvidconv > nvosd > nvvidconv2 > encoder > parser1 > mux > sink
    tiler.link(nvvidconv)
    nvvidconv.link(nvosd)
    nvosd.link(nvvidconv2)
    nvvidconv2.link(encoder)
    encoder.link(parser1)
    parser1.link(mux)
    mux.link(sink)
    
    loop = GLib.MainLoop() # Create a mainloop
    bus = pipeline.get_bus() # Retrieve the bus from the pipeline
    bus.add_signal_watch() # Add a watch for new messages on the bus
    bus.connect ("message", bus_call, loop) # Connect the loop to the callback function
    
    # We add probe to the sink pad of the tiler element, since by that time, 
    # the buffer would have had gotten all the metadata.

    # Retrieve the sink pad of the tiler
    tilersinkpad = tiler.get_static_pad("sink")
    if not tilersinkpad:
        sys.stderr.write(" Unable to get sink pad of tiler \n")

    # Add probe function to the sink pad of the tiler
    tilersinkpad.add_probe(Gst.PadProbeType.BUFFER, tiler_sink_pad_buffer_probe, 0)
    
    print("Starting pipeline \n")
    pipeline.set_state(Gst.State.PLAYING)
    try:
        loop.run()
    except:
        pass
    # cleaning up as the pipeline comes to an end
    pipeline.set_state(Gst.State.NULL)

Now, you can simply build and run the pipeline by calling this function (after picking a new output file name). Again, do not run the cell after until the pipeline runs to completion.

In [ ]:
output_file = "output2.mp4"
build_and_run_pipeline()

View the newly written file output. This should be identical to the first output video.

In [ ]:
Video("output2.mp4")

## Simple Customizations
Let's talk about a few simple ways to customize your new DeepStream Python application. Recall the file paths and additional options in [Step 4](#Step-4---Set-file-paths-and-additional-options).

### Change input sources
You can add any number of URIs to our input file list, and as you've seen, the application will create a source bin for each  input. The batch size properties for the streammux and the pgie will be set according to the number of sources, and the number of rows and columns for the tiler will be calculated and set accordingly. Note that the tiler will shrink/stretch each stream for the 2D tile to fit the total resolution.

Let's try a file list of 2 URIs.

Note: these URIs do not need to be the same as each other.

In [ ]:
input_file_list = ["file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264", \
                   "file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"]
                    # List of paths to input streams in URI format
output_file = "output3.mp4" # Output file location

In [ ]:
build_and_run_pipeline()

Observe the new tiled output with 2 streams.

In [ ]:
Video("output3.mp4")

### Disable the tracker and SGIEs
Through the `enable_tracker` setting, we can disable the tracker by setting the value to `0`, or `False`. Pipeline creation will then skip the creation/addition/linkage of the tracker and 3 SGIE elements.

In [ ]:
enable_tracker = 0  #Enable/disable tracker and SGIEs. 0 for disable, 1 for enable
output_file = "output4.mp4"

In [ ]:
build_and_run_pipeline()

Observe that SGIE inference no longer occurs, so information previously inferred from SGIEs (car make, color, type) are no longer generated and attached to the buffer, and thus do not appear in the OSD output

In [ ]:
Video("output4.mp4")

### Bounding box options
The probe function extracts the object level metadata attached to the buffer and can set the parameters of the bounding boxes by retrieving the NvOSDRectParams structure of the object meta. For example, the color of the border of the bounding box, whether or not the bounding box has a background color, and what that background color is. Originally, the bbox border was set to red, and there was no background color.

Let's see what happens when we set the border to blue, enable a background color, and set the background color to transparent red.

In [ ]:
# Note: for colors, each Red/Green/Blue/Alpha (RGBA) value should be between 0.0 and 1.0, inclusive.
# Bounding box options
bbox_border_color = {"R": 0.0, "G": 0.0, "B": 1.0, "A": 1.0} # Color of bounding box. Set to blue
bbox_has_bg_color = True # Bool for whether bounding box has background color
bbox_bg_color = {"R": 1.0, "G": 0.0, "B": 0.0, "A": 0.2} # Color of bbox background. Set to red with transparency

In [ ]:
output_file = "output5.mp4"
build_and_run_pipeline()

Observe the pretty new colors on our bounding boxes!

In [ ]:
Video("output5.mp4")

### Display text options
The display text refers to the text currently at the top left of each frame, with the content "Frame Number= ... Number of Objects = ...". This is created and attached to the frame meta by the probe function. We can change the x/y offset (position of the text with respect to the frame), the font of the text, font size, font color, whether the text box has a background, and the color of that background.

Let's move the text to the right, try a different, larger font, make the text green, and remove the background of the text box.

In [ ]:
# Display text options, to be added to the frame.
text_x_offset = 800 # Offset in the x direction where string should appear
text_y_offset = 15 # Offset in the y direction where string should appear
text_font_name = "Sans" # Font name
text_font_size = 16 # Font size
text_font_color = {"R" : 0.0, "G": 1.0, "B": 0.0, "A": 1.0} # Color of text font. Set to green
text_set_bg_color = False # Bool for whether text box has background color
text_bg_color = {"R": 0.0, "G": 0.0, "B": 0.0, "A": 1.0} # Color of text box background. Set to black

Before running the pipeline, change our input file list to use only one stream for clarity. 

In [ ]:
input_file_list = ["file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"] # List of paths to input streams in URI format

In [ ]:
output_file = "output6.mp4"
build_and_run_pipeline()

In [ ]:
Video("output6.mp4")

## Try it yourself!
Here are all of the options in one place again. Go ahead and make some changes, then run the pipeline and see how they affect the output!

In [ ]:
# File paths
input_file_list = ["file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264", \
                   "file:///opt/nvidia/deepstream/deepstream/samples/streams/sample_720p.h264"] 
                    # List of paths to input streams in URI format
output_file = "output7.mp4" # Output file location

# Tracker options
enable_tracker = 1 # Enable/disable tracker and SGIEs. 0 for disable, 1 for enable

# Note: for colors, each Red/Green/Blue/Alpha (RGBA) value should be between 0.0 and 1.0, inclusive.
# Bounding box options
bbox_border_color = {"R": 1.0, "G": 0.0, "B": 0.0, "A": 1.0} # Color of bounding box. Set to red
bbox_has_bg_color = False # Bool for whether bounding box has background color
bbox_bg_color = {"R": 1.0, "G": 0.0, "B": 0.0, "A": 0.2} # Color of bbox background. Set to red with transparency

# Display text options, to be added to the frame.
text_x_offset = 10 # Offset in the x direction where string should appear
text_y_offset = 12 # Offset in the y direction where string should appear
text_font_name = "Serif" # Font name
text_font_size = 10 # Font size
text_font_color = {"R" : 1.0, "G": 1.0, "B": 1.0, "A": 1.0} # Color of text font. Set to white
text_set_bg_color = True # Bool for whether text box has background color
text_bg_color = {"R": 0.0, "G": 0.0, "B": 0.0, "A": 1.0} # Color of text box background. Set to black

In [ ]:
build_and_run_pipeline()

In [ ]:
Video(output_file)

## Next Steps

Congratulations on completing your first DeepStream Python application! Please check out our [deepstream_python-apps](https://github.com/NVIDIA-AI-IOT/deepstream_python_apps) repository. There, you can explore some other use cases through our sample applications, and introduce yourself to the publicly available Python bindings code.